# Fintech Homework 6, ClockChain and CryptoCurrency

## NTU CSIE R08922024 胡安鳳 [922024]

Use the elliptic curve “secp256k1” as Bitcoin and Ethereum. Let G be
the base point in the standard. Let d be the last 6 digits of your student
ID number.

Module prequisities
pycoin ecdsa secp256k1 --> https://pycoin.readthedocs.io/en/latest/api.html
`pip3 install pycoin ecdsa`

### 1. Evaluate 4G

In [8]:
import numpy as np
from pycoin.ecdsa.secp256k1 import secp256k1_generator as g

STU_ID = 922024

x, y = 4 * g
print('x = %s, y = %s' %(hex(x), hex(y)))

x = 0xe493dbf1c10d80f3581e4904930b1404cc6c13900ee0758474fa94abe8c4cd13, y = 0x51ed993ea0d455b75642e2098ea51448d967ae33bfbdfe40cfe97bdc47739922


### 2. Evaluate 5G

In [9]:
x, y = 5 * g
print('x = %s, y = %s' %(hex(x), hex(y)))

x = 0x2f8bde4d1a07209355b4a7250a5c5128e88b84bddc619ab7cba8d569b240efe4, y = 0xd8ac222636e5e3d6d4dba9dda6c9c426f788271bab0d6840dca87d3aa6ac62d6


### 3. Evaluate 922024G

In [12]:
x, y = STU_ID * g
print('x = %s, y = %s' %(hex(x), hex(y)))

x = 0x4eb5558d3cefbc3aa76064f8a529e1992b6a2eae9c955d0869fc284fc6216db9, y = 0xb6a6a2108b1be93f13588bd3a41393ac88deb0e6f6ef9cb81ef00368cd7da92c


### 4. Double and Add Algorithm

In [30]:
list_operations = []
binary_str = ''
STU_ID = 922024
num = STU_ID


while num > 1:
    if num & 0x1 == 1:
        list_operations.append('a')
        binary_str += '1'
        num -= 1
    else:
        list_operations.append('d')
        binary_str += '0'
        num >>= 1
    
list_operations = list_operations[::-1]
binary_str = binary_str[::-1]

print('binary representation of 922024: ', binary_str)
print('add %d times ' % (list_operations.count('a')))
print('double %d times ' % (list_operations.count('d')))
print('total %d times' % (len(list_operations)))
print('datailed steps ', list_operations)

binary representation of 922024:  01010000010000101001001000
add 7 times 
double 19 times 
total 26 times
datailed steps  ['d', 'a', 'd', 'a', 'd', 'd', 'd', 'd', 'd', 'a', 'd', 'd', 'd', 'd', 'a', 'd', 'a', 'd', 'd', 'a', 'd', 'd', 'a', 'd', 'd', 'd']


### 5. Note that it is effortless to find -P from any P on a curve. If the addition of an inverse point is allowed, try your best to evaluate dG as fast as possible. Hint: 31P = 2(2(2(2(2P)))) - P.